In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import unidecode
import pymongo
import re

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/carter/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [3]:
browser.visit('http://quotes.toscrape.com/')

In [4]:
data = []
authors = set()
nextpage = 1
while(nextpage):
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    quotes = soup.find_all('div', class_='quote')
    for quote in quotes:
        tag_list=[]
        q = quote.find('span',class_='text').text
        author = quote.find('small',class_='author').text
        tags = quote.find_all('a', class_='tag')
        #Adds tags to list
        for tag in tags:
            tag_list.append(tag.text)
        authors.add(author)
    #Add data to dictionary to add to MongoDB later
        data.append({'author_name':author,'quote':q,'tags': tag_list})
    if browser.links.find_by_partial_text('Next '):
        browser.links.find_by_partial_text('Next ').click()
    else:
            nextpage=0

In [5]:
browser.quit()

In [6]:
author_data = []
for author in authors:
    #regex to replace any period(.) or space( ) with a hyphen(-), avoids putting in double hyphens(--) when there is a period and a space
    cleaned = unidecode.unidecode(re.sub(r'\. | |\.','-',author)).replace('\'','')
    author_ = cleaned if cleaned[-1] != '-' else cleaned[:-1]
    #scrapes quotes urls based on cleaned author name
    soup = BeautifulSoup(requests.get(f"http://quotes.toscrape.com/author/{author_}").text,'html.parser')
    born = soup.find('span',class_='author-born-date').text
    description = soup.find('div', class_='author-description').text.strip()
    author_data.append({'author_name':author,'born':born,'description':description})
                                                            

In [8]:
#MongoDB
#Adds data scraped above into MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client['quotes_db']
authors = db.authors
quotes = db.quotes
authors.insert_many(author_data)
quotes.insert_many(data)